In [2]:
# Copyright notice
__author__ = "Matteo Lulli"
__copyright__ = "Copyright (c) 2020-2021 Matteo Lulli (lullimat/idea.deploy), matteo.lulli@gmail.com"
__credits__ = ["Matteo Lulli"]
__license__ = """                                                                                                                                        
Permission is hereby granted, free of charge, to any person obtaining a copy                                                                             
of this software and associated documentation files (the "Software"), to deal                                                                            
in the Software without restriction, including without limitation the rights                                                                             
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell                                                                                
copies of the Software, and to permit persons to whom the Software is                                                                                    
furnished to do so, subject to the following conditions:                                                                                                 
                                                                                                                                                         
The above copyright notice and this permission notice shall be included in all                                                                           
copies or substantial portions of the Software.                                                                                                          
                                                                                                                                                         
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR                                                                               
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,                                                                                 
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE                                                                              
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER                                                                                   
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,                                                                            
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE                                                                            
SOFTWARE.                                                                                                                                                
"""
__version__ = "0.1"
__maintainer__ = "Matteo Lulli"
__email__ = "matteo.lulli@gmail.com"
__status__ = "Development"

# Tutorial: Two-dimensional Ising Model

Welcome to this tutorial on how to build a module for the two-dimensional Ising model using the *root* classes of the **idea.deploy** framework.

This tutorial will guide you in a step-by-step way into the process of developing a Metropolis Monte Carlo simulation for the two-dimensional Ising model. This will be done with different degrees of complication

- starting first from a simple implementation where spin variables are represented by a signed integer (```int```), 
- then shifting to using unsigned chars (```unsigned char```), and 
- finally land on an asynchronous-multi-spin-coded (aMSC) implementation where the spin variables will be stored in the bits of unsigned integers (```unsigned int``` or ```unsigned``` according to the architecture) and will be updated in parallel. 

The last step allows for roughly a 32-fold factor gain in performance with respect to the simples signed integer representation since we will be simulating 32 systems in paraller. This is done at the cost of introducing a small correlation between different realizations of the system. At the same time, this process will allow to have a faster convergence of the statistical analysis.

During this process we will shift strategy for the memory addressing going from a straightforward geometrical implementation which will then evolve to a checkerboard separation and finally end in a **sliced** scheme which allows for a better alignement of the memory reads while halving the number of MPI memory transfers in the multi-process implementation.

We will be measuring the performance differences with the built-in tools of the **idea.deploy** framework and also develop some standard statistical analysis tools along the way.

This tutorial is clearly *physics-oriented* and serves the double purpose of illustrating the functionalities of the framework while being *educational* as far as the two-dimensional Ising model and simple statistical analysis are concerned.

I will be constantly pushing updates on the *develop* branch of this project so that the last version can simply be retreived by constantly pulling the modifications to the repository.

I will also keep the tutorial finely grained in such a way that each subsection is self-consistent with each new class being a copy of the previous one and simply distinguished by a ```V#``` suffix.

In [22]:
# Import Statements

'''
Appending the root directory of the idea.deploy project.
This is done explicitly here. 
In the modules this is done in the local __init__.py file
'''
import sys
sys.path.append("../../")

'''
Importing languages types
'''
from idpy.IdpyCode import CUDA_T, OCL_T, IDPY_T

'''
Importing the congruential pseudo-random number generators
'''
from idpy.PRNGS.CRNGS import CRNGS


In [23]:
_test_crng = CRNGS(lang = OCL_T, cl_kind = 'cpu', n_prngs = 2 ** 7)

OpenCL:  Device: 0 Apple M1 Memory:17179869184
uint32


In [24]:
_test_crng.Norm(rand_type = {'RANDType': 'float'})

TypeError: string indices must be integers

In [6]:
_test_crng.sims_idpy_memory['seeds']

cl.Array([1791095846], dtype=uint32)

In [17]:
_test_crng.End()